In [11]:
import sys, time, pickle
from pyspark import SparkContext, SparkConf
from pyspark.sql import *
from pyspark.sql.types import *
from IPython.display import display, HTML
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pymongo import MongoClient
import pandas as pd
import numpy as np

from sklearn.feature_selection import mutual_info_regression

import matplotlib.pyplot as plt

# make matplotlib plot sizes larger
plt.rcParams['figure.figsize'] = [30, 20]

conf = SparkConf().setAppName('Steam Random Forest Regressor').setMaster('spark://sparkmaster:7077')
SparkContext.setSystemProperty('spark.executor.memory', '2g') # memory per executor
SparkContext.setSystemProperty('spark.executor.cores', '6') # cores per executor
SparkContext.setSystemProperty('spark.executor.instances', '3') # per worker (computer)

# https://spark.apache.org/docs/3.0.0-preview/configuration.html#dynamic-allocation
# https://stackoverflow.com/questions/26168254/how-to-set-amount-of-spark-executors
# https://blog.cloudera.com/how-to-tune-your-apache-spark-jobs-part-2/

# SparkContext.setSystemProperty("spark.shuffle.service.enabled", "True") # required for dynamic allocation below
# SparkContext.setSystemProperty("spark.dynamicAllocation.enabled", "True")
# SparkContext.setSystemProperty("spark.executor.cores", "4")
# SparkContext.setSystemProperty("spark.dynamicAllocation.minExecutors", "1")
# SparkContext.setSystemProperty("spark.dynamicAllocation.maxExecutors", "5")
# SparkContext.setSystemProperty('spark.executor.memory', '2g') # memory per executor

sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

/spark/python/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [2]:
# Customizable Variables

# what percentage discount should be predicted?
percentage_discount_predict = 25

In [3]:
sc._conf.getAll()

[('spark.executor.memory', '2g'),
 ('spark.driver.host', 'jupyterlab'),
 ('spark.executor.instances', '3'),
 ('spark.app.name', 'Steam Random Forest Regressor'),
 ('spark.app.startTime', '1644967979376'),
 ('spark.executor.id', 'driver'),
 ('spark.master', 'spark://sparkmaster:7077'),
 ('spark.app.id', 'app-20220215163301-0001'),
 ('spark.driver.port', '41361'),
 ('spark.executor.cores', '6'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true')]

In [4]:
client = MongoClient(host="192.168.1.140", port=27017)
db = client['steam']
apps = db['apps']
history = db['pricehistory']
opencritic = db['opencritic']
topgames = db['topgames']
twitch = db['twitchhistorical']

# TODO: account for cases where there are multiple publishers or developers?
ret = apps.aggregate([
    {"$match": {"updated_date": {"$exists": True}, "type": {"$in": ["game", "dlc"]}, "release_date.coming_soon": {"$ne": True}, "is_free": False}},
    {"$project": {"_id":0, "name":1, "appid":1, "release_date": "$release_date.date", "total_recommendations": "$recommendations.total",
                  "initial_price": "$price_overview.initial", "metacritic_score": "$metacritic.score", "type":1, "required_age":1,
                 "windows":"$platforms.windows", "mac":"$platforms.mac", "linux":"$platforms.linux",
                 "publisher": {"$arrayElemAt": ["$publishers", 0]}, "developer": {"$arrayElemAt": ["$developers", 0]},
                 "number_dlc": {'$size': {"$ifNull": [ "$dlc", [] ]}}, "number_genres": {'$size': {"$ifNull": [ "$genres", [] ]}}, "number_categories": {'$size': {"$ifNull": [ "$categories", [] ]}}}}
])

In [5]:
# save as pandas dataframe
df = pd.DataFrame(ret)

In [6]:
df

,type,name,required_age,appid,release_date,initial_price,metacritic_score,windows,mac,linux,publisher,developer,number_dlc,number_genres,number_categories,total_recommendations
0,game,Legend: Hand of God,0,12630,"May 5, 2009",999.0,57.0,True,False,False,DTP Entertainment,Master Creating,0,2,1,NaN
1,game,realMyst,0,63600,"Aug 4, 2010",599.0,66.0,True,False,False,Cyan Worlds,Cyan Worlds,0,2,1,231.0
2,dlc,Crusader Kings II: African Portraits,0,210897,"Oct 16, 2012",199.0,NaN,True,True,True,Paradox Interactive,Paradox Interactive,0,1,3,NaN
3,game,Planetary Annihilation,0,233250,"Sep 5, 2014",2999.0,62.0,True,True,True,Planetary Annihilation Inc,Planetary Annihilation Inc,2,1,6,14931.0
4,dlc,Mark of the Ninja: Special Edition DLC,17,239570,"Aug 16, 2013",499.0,NaN,True,True,True,,Klei Entertainment,0,3,8,239.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86619,game,Garrison Gauntlet,0,912360,2018-08-17 00:00:00,99.0,NaN,True,False,False,Ryan Silberman,Ryan Silberman,0,2,1,NaN
86620,game,Battle Realms: Zen Edition,0,1025600,2019-12-03 00:00:00,999.0,82.0,True,False,False,Ed Del Castillo,Ed Del Castillo,0,3,6,1836.0
86621,dlc,We Happy Few - We All Fall Down,17,1024770,2019-11-19 00:00:00,799.0,NaN,True,False,False,Gearbox Publishing,Compulsion Games,0,3,6,NaN
86622,game,Bombing Quest,0,1018860,2020-03-04 00:00:00,1499.0,NaN,True,False,False,Team Puh,Team Puh,0,4,4,NaN


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86624 entries, 0 to 86623
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   type                   86624 non-null  object 
 1   name                   86624 non-null  object 
 2   required_age           86624 non-null  object 
 3   appid                  86624 non-null  int64  
 4   release_date           86624 non-null  object 
 5   initial_price          79834 non-null  float64
 6   metacritic_score       3820 non-null   float64
 7   windows                86624 non-null  bool   
 8   mac                    86624 non-null  bool   
 9   linux                  86624 non-null  bool   
 10  publisher              86624 non-null  object 
 11  developer              85067 non-null  object 
 12  number_dlc             86624 non-null  int64  
 13  number_genres          86624 non-null  int64  
 14  number_categories      86624 non-null  int64  
 15  to

In [8]:
# fix cases where it lists 'age+'
df['required_age'] = df['required_age'].replace(['17+'],'17')
df['required_age'] = df['required_age'].replace(['7+'],'7')

# convert column to datetime
df['release_date'] = pd.to_datetime(df['release_date'])

# convert column to int64 type
df['required_age'] = pd.to_numeric(df['required_age'])

ValueError: Unable to parse string "7+" at position 47449

In [ ]:
# figure out the release day in the year
df['release_day_in_year'] = df['release_date'].dt.dayofyear

In [ ]:
# aggregate the steam recommendations and tie them in

ret = apps.aggregate([
    {"$unwind":"$reviews"},
    {"$unwind":"$reviews.rollups"},
    {
        "$group": {
            "_id": "$appid",
           "sum_recommendations_up": {"$sum": "$reviews.rollups.recommendations_up"},
            "sum_recommendations_down": {"$sum": "$reviews.rollups.recommendations_down"}
        }
    }
])

# save as pandas dataframe
recommendations = pd.DataFrame(ret)
recommendations = recommendations.rename(columns={"_id": "appid"})

recommendations.info()

In [ ]:
df = pd.merge(df, recommendations, on="appid", how="left")
del recommendations

In [ ]:
df

In [ ]:
df.info()

In [ ]:
ret = history.aggregate([
    {"$match": {"discount_percent": {"$gte": percentage_discount_predict}}},
    {"$group": {"_id": "$appid", "date": {"$min": "$date"}}},
    {"$project": {"_id":0, "appid":"$_id", "date":1}}
])

In [ ]:
history_df = pd.DataFrame(ret)

In [ ]:
history_df

In [ ]:
history_df.info()

In [ ]:
df = pd.merge(df, history_df, on="appid", how="left")
del history_df

In [ ]:
# find time in days between release and at least X% off
df['days_until_discount'] = df['date'] - df['release_date']
df['days_until_discount'] = df['days_until_discount'] / np.timedelta64(1,"D")

In [ ]:
df

In [ ]:
# get top games information

ret = topgames.aggregate([
    {"$match": {}},
    {"$project": {"_id":0, "date":1, "appid":1, "currentplayers":1, "peaktoday":1}}
])
tgames = pd.DataFrame(ret)
tgames['appid'] = pd.to_numeric(tgames['appid'])

tgames = pd.merge(tgames, df[["appid", "release_date"]], on="appid", how="left").dropna()

tgames

In [ ]:
tgames.info()

In [ ]:
# find difference between entry date and release date
tgames['days_since_release'] = (tgames['date'] - tgames['release_date']).dt.days

#tgames.query("appid==1097150 & days_since_release>=0 & days_since_release<=30")
tgames

In [ ]:
bins = range(-1, 7, 1)
groups = tgames.groupby(['appid', pd.cut(tgames.days_since_release, bins, labels=["start_players_{:01d}".format(x) for x in bins[:-1]])])
groups = groups.mean().currentplayers.unstack()
groups = groups.fillna(0)

#groups[groups.index.isin([1097150])]
groups

In [ ]:
df = pd.merge(df, groups, left_on="appid", right_index=True, how="left")
del groups
del tgames

df

In [ ]:
df.info()

In [ ]:
# get twitch games

ret = twitch.aggregate([
    {"$match": {}},
    {"$project": {"_id":0, "date":1, "name":1, "gamerank":1, "viewer_count":1}}
])
twitch_df = pd.DataFrame(ret)

twitch_df = pd.merge(twitch_df, df[["name", "release_date"]], on="name", how="left").dropna()

twitch_df

In [ ]:
twitch_df.info()

In [ ]:
# find difference between entry date and release date
twitch_df['days_since_release'] = (twitch_df['date'] - twitch_df['release_date']).dt.days

#twitch_df.query("appid==1097150 & days_since_release>=0 & days_since_release<=30")
twitch_df

In [ ]:
bins = range(-1, 7, 1)
groups_viewers = twitch_df.groupby(['name', pd.cut(twitch_df.days_since_release, bins, labels=["start_twitch_viewers_{:01d}".format(x) for x in bins[:-1]])])
groups_viewers = groups_viewers.mean().viewer_count.unstack()
groups_viewers = groups_viewers.fillna(0)

groups_viewers

In [ ]:
bins = range(-1, 7, 1)
groups_rank = twitch_df.groupby(['name', pd.cut(twitch_df.days_since_release, bins, labels=["start_twitch_rank_{:01d}".format(x) for x in bins[:-1]])])
groups_rank = groups_rank.mean().gamerank.unstack()
groups_rank = groups_rank.fillna(0)

groups_rank

In [ ]:
# merge twitch viewership and gamerank back into master df
df = pd.merge(df, groups_viewers, left_on="name", right_index=True, how="left")
del groups_viewers
df = pd.merge(df, groups_rank, left_on="name", right_index=True, how="left")
del groups_rank

del twitch_df

In [ ]:
# fill in all the NaN spots
players_filter_col = [c for c in df.columns if c.startswith('start')]
print(players_filter_col)
df[players_filter_col] = df[players_filter_col].fillna(0)

In [ ]:
df

In [ ]:
df.info()

In [ ]:
ret = opencritic.aggregate([
    {"$match": {"steamId": {"$exists": True, "$nin": [None, "", "null", "NaN"], "$not": {"$type": 10}}}},
    {"$project": {"_id":0, "appid":"$steamId", "game_type":"$type", "hasLootBoxes":1, "isMajorTitle":1, "medianScore":1, "numReviews":1,
                  "numTopCriticReviews":1, "percentRecommended":1, "percentile":1, "tier":1, "topCriticScore":1,
                 "number_platforms": {'$size': {"$ifNull": [ "$Platforms", [] ]}}, "number_skus": {'$size': {"$ifNull": [ "$Skus", [] ]}}}}
])

In [ ]:
opencritic_df = pd.DataFrame(ret)

# fix cases where we don't have data
opencritic_df = opencritic_df.replace({'medianScore': -1, 'percentRecommended': -1, 'percentile': -1, 'topCriticScore': -1}, np.nan)

opencritic_df

In [ ]:
opencritic_df.describe()

In [ ]:
# convert object to int64 type
opencritic_df["appid"] = pd.to_numeric(opencritic_df["appid"])

In [ ]:
# NaNs will be pruned out later, in order to use as much of the sparse OpenCritic data that we have, we fill it in with zeros
opencritic_df.fillna(0, inplace=True)

In [ ]:
opencritic_df

In [ ]:
opencritic_df.info()

In [ ]:
df = pd.merge(df, opencritic_df, on="appid", how="left")
del opencritic_df

In [ ]:
df

In [ ]:
# https://analyticsindiamag.com/5-ways-handle-missing-values-machine-learning-datasets/
# TODO: use mean or median for cases where we have missing data



# just assume anything not filled in doesn't have loot boxes
#df = df.replace({'hasLootBoxes': np.nan}, False)

In [ ]:
df.info()

In [ ]:
# we can't run prediction with empty values
# make a copy so we can continue modifying the dataframe
df = df.dropna().copy()

In [ ]:
df

In [ ]:
df.info()

In [ ]:
feature_list = ['required_age', 'metacritic_score', 'sum_recommendations_up', 'sum_recommendations_down', 'total_recommendations', 'initial_price', 'windows_num',
                'mac_num', 'linux_num', 'developer_num', 'publisher_num',
               'number_dlc', 'release_day_in_year', 'number_genres', 'number_categories']

# items from OpenCritic
feature_list = feature_list + ['type_num', 'hasLootBoxes_num', 'isMajorTitle_num',
            'tier_num', 'game_type_num', 'medianScore', 'numReviews', 'numTopCriticReviews', 'percentRecommended', 'percentile',
            'topCriticScore', 'number_platforms', 'number_skus']

feature_list = feature_list + players_filter_col
print(feature_list)

assembler = VectorAssembler(inputCols=feature_list, outputCol="features")

In [ ]:
# https://www.silect.is/blog/2019/4/2/random-forest-in-spark-ml
# an example Random Forest in Spark

In [ ]:
# encode columns to numeric so we can run regression(s)

# items from OpenCritic
df['type_num'] = pd.factorize(df['type'])[0]
df['hasLootBoxes_num'] = pd.factorize(df['hasLootBoxes'])[0]
df['isMajorTitle_num'] = pd.factorize(df['isMajorTitle'])[0]
df['tier_num'] = pd.factorize(df['tier'])[0]
df['game_type_num'] = pd.factorize(df['game_type'])[0]

df['windows_num'] = pd.factorize(df['windows'])[0]
df['mac_num'] = pd.factorize(df['mac'])[0]
df['linux_num'] = pd.factorize(df['linux'])[0]
df['developer_num'] = pd.factorize(df['developer'])[0]
df['publisher_num'] = pd.factorize(df['publisher'])[0]

In [ ]:
df

In [ ]:
df.info()

In [ ]:
# TODO: qq plot, residuals, etc.
# https://github.com/tirthajyoti/Machine-Learning-with-Python/blob/master/Regression/Regression_Diagnostics.ipynb
# https://towardsdatascience.com/how-do-you-check-the-quality-of-your-regression-model-in-python-fa61759ff685


In [ ]:
# perform k-means clustering to create an additional numeric column for feature engineering
# https://www.kaggle.com/ryanholbrook/clustering-with-k-means
# https://spark.apache.org/docs/latest/ml-clustering.html

# https://stackoverflow.com/questions/37513355/converting-pandas-dataframe-into-spark-dataframe-error
schema = StructType([ StructField("type", StringType(), True)\
                       ,StructField("name", StringType(), True)\
                       ,StructField("required_age", IntegerType(), True)\
                       ,StructField("appid", IntegerType(), True)\
                       ,StructField("release_date", DateType(), True)\
                       ,StructField("initial_price", DoubleType(), True)\
                       ,StructField("metacritic_score", DoubleType(), True)\
                       ,StructField("windows", BooleanType(), True)\
                       ,StructField("mac", BooleanType(), True)\
                       ,StructField("linux", BooleanType(), True)\
                       ,StructField("publisher", StringType(), True)\
                       ,StructField("developer", StringType(), True)\
                       ,StructField("number_dlc", IntegerType(), True)\
                       ,StructField("number_genres", IntegerType(), True)\
                       ,StructField("number_categories", IntegerType(), True)\
                       ,StructField("total_recommendations", DoubleType(), True)\
                       ,StructField("release_day_in_year", DoubleType(), True)\
                       ,StructField("sum_recommendations_up", DoubleType(), True)\
                       ,StructField("sum_recommendations_down", DoubleType(), True)\
                       ,StructField("date", DateType(), True)\
                       ,StructField("days_until_discount", DoubleType(), True)\
                       ,StructField("start_players_-1", DoubleType(), True)\
                       ,StructField("start_players_0", DoubleType(), True)\
                       ,StructField("start_players_1", DoubleType(), True)\
                       ,StructField("start_players_2", DoubleType(), True)\
                       ,StructField("start_players_3", DoubleType(), True)\
                       ,StructField("start_players_4", DoubleType(), True)\
                       ,StructField("start_players_5", DoubleType(), True)\
                       ,StructField("start_twitch_viewers_-1", DoubleType(), True)\
                       ,StructField("start_twitch_viewers_0", DoubleType(), True)\
                       ,StructField("start_twitch_viewers_1", DoubleType(), True)\
                       ,StructField("start_twitch_viewers_2", DoubleType(), True)\
                       ,StructField("start_twitch_viewers_3", DoubleType(), True)\
                       ,StructField("start_twitch_viewers_4", DoubleType(), True)\
                       ,StructField("start_twitch_viewers_5", DoubleType(), True)\
                       ,StructField("start_twitch_rank_-1", DoubleType(), True)\
                       ,StructField("start_twitch_rank_0", DoubleType(), True)\
                       ,StructField("start_twitch_rank_1", DoubleType(), True)\
                       ,StructField("start_twitch_rank_2", DoubleType(), True)\
                       ,StructField("start_twitch_rank_3", DoubleType(), True)\
                       ,StructField("start_twitch_rank_4", DoubleType(), True)\
                       ,StructField("start_twitch_rank_5", DoubleType(), True)\
                       ,StructField("hasLootBoxes", StringType(), True)\
                       ,StructField("isMajorTitle", StringType(), True)\
                       ,StructField("medianScore", DoubleType(), True)\
                       ,StructField("numReviews", DoubleType(), True)\
                       ,StructField("numTopCriticReviews", DoubleType(), True)\
                       ,StructField("percentRecommended", DoubleType(), True)\
                       ,StructField("percentile", DoubleType(), True)\
                       ,StructField("tier", StringType(), True)\
                       ,StructField("topCriticScore", DoubleType(), True)\
                       ,StructField("game_type", StringType(), True)\
                       ,StructField("number_platforms", DoubleType(), True)\
                       ,StructField("number_skus", DoubleType(), True)\
                       ,StructField("type_num", IntegerType(), True)\
                       ,StructField("hasLootBoxes_num", IntegerType(), True)\
                       ,StructField("isMajorTitle_num", IntegerType(), True)\
                       ,StructField("tier_num", IntegerType(), True)\
                       ,StructField("game_type_num", IntegerType(), True)\
                       ,StructField("windows_num", IntegerType(), True)\
                       ,StructField("mac_num", IntegerType(), True)\
                       ,StructField("linux_num", IntegerType(), True)\
                       ,StructField("developer_num", IntegerType(), True)\
                       ,StructField("publisher_num", IntegerType(), True)])

# convert pandas dataframe to spark dataframe
temp_df = sqlContext.createDataFrame(df, schema=schema)
temp_df.printSchema()

dataset = assembler.transform(temp_df)
dataset.select("features").show(truncate=False)

cost = np.ones(17)
for k in range(2,17):
    # Trains a k-means model.
    kmeans = KMeans(k=k).setSeed(1)
    model = kmeans.fit(dataset)
    
    # Make predictions
    predictions = model.transform(dataset)

    # Evaluate clustering by computing Silhouette score
    evaluator = ClusteringEvaluator()

    silhouette = evaluator.evaluate(predictions)
    print("k: " + str(k) + " - Silhouette with squared euclidean distance = " + str(silhouette))
    
    cost[k] = silhouette

    # Shows the result.
    centers = model.clusterCenters()
    #print("Cluster Centers: ")
    #for center in centers:
        #print(center)
    
del temp_df

In [ ]:
plt.plot(np.arange(1,18), cost, linestyle='solid')
plt.ylabel('Silhouette cost')
plt.xlabel('k clusters')
plt.title('Cost from changing k clusters')

In [ ]:
# Trains a k-means model.
kmeans = KMeans(k=10).setSeed(1)
model = kmeans.fit(dataset)

# Make predictions
predictions = model.transform(dataset)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

cost[k] = silhouette

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)
    
cluster_predictions = predictions.toPandas()

In [ ]:
cluster_predictions

In [ ]:
cluster_predictions['prediction'].unique()

In [ ]:
df['cluster_prediction'] = cluster_predictions['prediction'].values

In [ ]:
df['cluster_prediction'].unique()

In [ ]:
feature_list = feature_list + ['cluster_prediction']
print(feature_list)

In [ ]:
# persist and serialize objects to disk for next step
pickle.dump(df, open("/work/df.p", "wb"))
pickle.dump(feature_list, open("/work/feature_list.p", "wb"))

In [10]:
sc.stop()